Открываем файл с текстом:

In [133]:
with open ('korosteleva_carmarthen.txt') as f:
  data = f.read()

Мы знаем, что истории внутри книги разделены тремя звёздочками. А еще обрежем первые 43 символа -- это название книги и автор. Обрежем также примечания к тексту -- они начинаются после фразы "Август 2000 г."

In [134]:
chapters = data[44:].split('* * *')
chapters[-1] = chapters[-1].split('Август 2000 г')[0]

# Извлечение имен
Теперь для каждой истории извлечем все имена, которые там встречаются

Для этого воспользуемся лингвистической библиотекой **Natasha**. Наташа -- безумно умная библиотека, которая в числе прочего умеет извлекать из текста именованные сущности -- названия людей, географический объектов и так далее.


> Наташа -- это нейросетевая модель, обученная на новостях с размеченными именованными сущностями; поэтому она отлично справляется с новостями, но и с другими жанрами она справляется тоже неплохо.


*(Хотя с нашими валлийскими и не только именами она помучается..))))*

### Сначала технические шаги:
* устанавливаем библиотеку
* импортируем нужные блоки


In [7]:
!pip install natasha

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 92.8 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=ffdb767a27f6d38c76e8ad602928706da87d211142186e9753f0b3dfe275dd7c
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26097 sha256=503f8a19bceaad4fc4a1a6ba2ea0393e9936de5fd00b6586fa65e5aea01ce125
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built docopt intervaltree


In [135]:
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsNERTagger,

    PER,
    NamesExtractor,

    Doc
)

segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
ner_tagger = NewsNERTagger(emb)

morph_vocab = MorphVocab()
names_extractor = NamesExtractor(morph_vocab)

Посмотрим, как Наташа извлекает имена

In [136]:
text = 'Афина приехала в подмосковное Пущино на школу Сова. Она знакомится с библиотекой Наташа'
doc = Doc(text)

doc.segment(segmenter)
doc.tag_morph(morph_tagger)

doc.tag_ner(ner_tagger)
for span in doc.spans:
    print(span)

DocSpan(stop=5, type='PER', text='Афина', tokens=[...])
DocSpan(start=30, stop=36, type='LOC', text='Пущино', tokens=[...])
DocSpan(start=46, stop=50, type='LOC', text='Сова', tokens=[...])
DocSpan(start=81, stop=87, type='PER', text='Наташа', tokens=[...])


Дальше определим функцию, которая берет главу, и извлекает из нее названия персонажей

In [54]:
from tqdm import tqdm

In [137]:
def NER(text):
    '''
    функция, извлекающая имена
    input: глава
    output: список имен в этой главе
    '''
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.tag_ner(ner_tagger)
    for s in doc.spans:
        s.tokens[-1].lemmatize(morph_vocab)
    return [(s.tokens[-1].lemma, s.type) for s in doc.spans]


def get_names(book):
    '''
    функция, которая перебирает главы и записывает, сколько раз персонажи встретились вместе в одной главе
    input: list глав
    output: словарь имен и словарь связей
    '''
    persons = {} # все имена
    connect = {} # связи

    for art in tqdm(book):
        nfacts = NER(art) # извлекаем имена
        # Так как один участник может упоминаться несколько раз, строим список с единственными упоминаниями.
        nam = [fact[0].split(" ")[-1] for fact in nfacts if fact[1] == 'PER']
        snam = list(set(nam))
        # Пробрасываем связи между людьми. Главное - не писать сколько раз человек связан между собой.
        for n in snam:
            persons[n] = persons.get(n, 0) + 1
            pers = connect.get(n, {})
            for n2 in snam:
                if n != n2:
                    pers[n2] = pers.get(n2, 0)+1
            connect[n] = pers
    return persons, connect

Извлечем связи из книги:

In [138]:
names, connections = get_names(chapters)

100%|██████████| 211/211 [00:49<00:00,  4.23it/s]


### Теперь давайте введем несколько ограничений:
* оставим только самые частотные имена -- такие, которые встречаются в целом по книге больше 10 раз (`names[n2] > 10`)
* оставим только персонажей, "взаимодействовавших" (то есть встречавшихся в одной главе) больше пяти раз (`connections[n][n2] > 5`)
* удалим всякие перлы Наташи: например, она лемматизирует Змейка как *змейка*, а Керидвен -- как *керидвена*; а еще считает, что скобка -- это имя собственное (`n2 != ')' and n2 != 'змейка' and n2 != 'керидвена'`)



In [141]:
pers2 = {
    n: {
        n2: connections[n][n2]
        for n2 in connections[n].keys()
        if connections[n][n2] > 5 and names[n2] > 10 and n2 != ')' and n2 != 'змейка' and n2 != 'керидвена'
    }
    for n in connections.keys()
    if names[n] > 10 and n != ')' and n != 'змейка' and n != 'керидвена'
}


In [ ]:
pers2

И запишем все в CSV-файл, чтобы потом скормить его Gephi.

In [140]:
import csv

with open('character_interactions.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Source', 'Target', 'Weight'])  # такие заголовки требуются для gephi

    for person1, interactions in pers2.items():
        for person2, weight in interactions.items():
            writer.writerow([person1, person2, weight])